In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 1 #当日は0、前日は1・・・
date2 = "08/18"
pmc = 22011043 # 店舗ID
SPREADSHEET_KEY = '1SAAVEZNpyXmPW_gNKJZbcG-UC8m5IU2h-oaQqTyFz60' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 281 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+456@gmail.com"
user_pw = "paruparu22"

Template_p_path = "site777/template_p.png"
Template_s_path = "site777/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
chrome_service = ChromeService(executable_path='C:\\Users\\garden026\\WD\\114.0.5735.90\\chromedriver.exe')

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "555":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "555":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/18', '025677', '1', '23590.0', '-14143.0', '4', '2']
['08/18', '025677', '2', '26410.0', '-17754.0', '3', '2']
['08/18', '025677', '3', '15641.0', '-8726.0', '3', '2']
['08/18', '025677', '4', '18718.0', '-14444.0', '1', '1']
['08/18', '025677', '5', '7436.0', '7524.0', '10', '1']
['08/18', '025677', '6', '22821.0', '-6920.0', '10', '4']
['08/18', '025677', '7', '13077.0', '-9930.0', '1', '1']
['08/18', '025677', '8', '15128.0', '-3911.0', '6', '4']
['08/18', '025677', '9', '12564.0', '8126.0', '14', '5']
['08/18', '025677', '10', '48718.0', '12941.0', '36', '8']
['08/18', '025677', '11', '15897.0', '1806.0', '11', '1']
['08/18', '025677', '12', '7179.0', '1806.0', '5', '3']
['08/18', '025677', '13', '27436.0', '-15045.0', '5', '3']
['08/18', '025677', '14', '25128.0', '-16550.0', '3', '2']
['08/18', '025677', '15', '12821.0', '1506.0', '8', '1']
['08/18', '025606', '16', '15128.0', '-2106.0', '7', '1']
['08/18', '025606', '17', '3077.0', '-2406.0', '0', '0']
['08/18', '025606', '

['08/18', '025779', '144', '7436.0', '-6018.0', '0', '0']
['08/18', '025545', '145', '19231.0', '10232.0', '29', '7']
['08/18', '025847', '146', '6923.0', '-6018.0', '2', '2']
['08/18', '025652', '147', '23846.0', '17154.0', '46', '4']
['08/18', '025722', '148', '11026.0', '-902.0', '13', '4']
['08/18', '025841', '149', '18718.0', '-5717.0', '15', '6']
['08/18', '025844', '150', '8462.0', '-5416.0', '3', '2']
['08/18', '025501', '151', '16154.0', '-902.0', '24', '5']
['08/18', '025775', '152', '12821.0', '2709.0', '7', '2']
['08/18', '025874', '153', '13333.0', '-10832.0', '1', '1']
['08/18', '025874', '154', '7692.0', '603.0', '13', '2']
['08/18', '025883', '155', '19744.0', '302.0', '23', '8']
['08/18', '025883', '156', '3333.0', '-1504.0', '4', '4']
['08/18', '025891', '157', '30513.0', '16853.0', '71', '25']
['08/18', '025891', '158', '35128.0', '-20161.0', '21', '14']
['08/18', '025897', '159', '37692.0', '22269.0', '44', '10']
['08/18', '025897', '160', '43077.0', '-26481.0', '12

['08/18', '025575', '615', '24615.0', '4816.0', '47', '11']
['08/18', '025835', '616', '35385.0', '302.0', '99', '18']
['08/18', '025722', '617', '29744.0', '-9027.0', '21', '11']
['08/18', '025613', '618', '22564.0', '13543.0', '35', '35']
['08/18', '025466', '619', '35897.0', '603.0', '44', '16']
['08/18', '025835', '620', '46667.0', '-7221.0', '99', '25']
['08/18', '025514', '621', '25897.0', '5418.0', '38', '8']
['08/18', '025572', '622', '41282.0', '-5416.0', '45', '15']
['08/18', '025440', '623', '36154.0', '-9930.0', '24', '9']
['08/18', '025465', '624', '35897.0', '-1805.0', '41', '15']
['08/18', '025793', '625', '23590.0', '-10832.0', '21', '8']
['08/18', '025735', '626', '25128.0', '8126.0', '29', '8']
['08/18', '025597', '627', '28974.0', '-1805.0', '39', '39']
['08/18', '025559', '628', '42821.0', '8728.0', '55', '20']
['08/18', '025509', '629', '29744.0', '904.0', '36', '16']
['08/18', '025727', '630', '20769.0', '-10231.0', '10', '10']
['08/18', '025722', '631', '39231.0'

['08/18', '025817', '752', '8462.0', '-3610.0', '12', '1']
['08/18', '025817', '753', '28718.0', '-3309.0', '64', '14']
['08/18', '025836', '754', '1026.0', '-902.0', '0', '0']
['08/18', '025836', '755', '256.0', '-902.0', '0', '0']
['08/18', '025836', '756', '256.0', '-902.0', '0', '0']
['08/18', '025737', '757', '4872.0', '-3911.0', '0', '0']
['08/18', '025726', '758', '1795.0', '-1203.0', '0', '0']
['08/18', '025879', '759', '2564.0', '-1504.0', '1', '1']
['08/18', '025879', '760', '513.0', '-902.0', '0', '0']
['08/18', '025879', '761', '6154.0', '-4814.0', '0', '0']
['08/18', '025879', '762', '1282.0', '4515.0', '6', '1']
['08/18', '025879', '763', '4103.0', '-1805.0', '2', '1']
['08/18', '025553', '764', '2564.0', '-2106.0', '0', '0']
['08/18', '025553', '765', '1795.0', '-1504.0', '0', '0']
['08/18', '025553', '766', '1282.0', '-902.0', '0', '0']
['08/18', '025553', '767', '2051.0', '-1805.0', '0', '0']
['08/18', '025553', '768', '3590.0', '-3008.0', '0', '0']
['08/18', '025553',

['08/18', '120096', '318', '3548', '-93.0', '14', '8', '--']
['08/18', '120096', '319', '3376', '93.0', '15', '7', '--']
['08/18', '120096', '320', '6916', '-509.0', '21', '23', '--']
['08/18', '120010', '321', '6277', '-648.0', '24', '13', '--']
['08/18', '120010', '322', '5196', '93.0', '22', '8', '--']
['08/18', '120010', '323', '6637', '-139.0', '22', '21', '--']
['08/18', '120010', '324', '4896', '278.0', '21', '12', '--']
['08/18', '120010', '325', '6888', '1157.0', '29', '25', '--']
['08/18', '120010', '326', '5605', '-556.0', '20', '14', '--']
['08/18', '120010', '327', '7041', '-1435.0', '20', '22', '--']
['08/18', '120010', '328', '3954', '-833.0', '12', '8', '--']
['08/18', '120010', '329', '6205', '-648.0', '19', '24', '--']
['08/18', '120010', '330', '6218', '-1852.0', '18', '12', '--']
['08/18', '120010', '331', '5576', '-324.0', '21', '13', '--']
['08/18', '120010', '332', '4547', '324.0', '20', '8', '--']
['08/18', '120010', '333', '7213', '-278.0', '23', '23', '--']
['

['08/18', '120108', '452', '1905', '-1343.0', '0', '0', '5']
['08/18', '120108', '453', '4396', '972.0', '0', '0', '32']
['08/18', '120108', '454', '1309', '-1111.0', '0', '0', '3']
['08/18', '120149', '455', '5400', '-1111.0', '0', '16', '59']
['08/18', '120149', '456', '1561', '2037.0', '0', '3', '29']
['08/18', '120149', '457', '3022', '-1713.0', '0', '8', '25']
['08/18', '120149', '458', '4816', '-324.0', '0', '13', '56']
['08/18', '120110', '459', '1678', '1620.0', '16', '8', '--']
['08/18', '120110', '460', '765', '-370.0', '3', '1', '--']
['08/18', '120110', '461', '3108', '-2361.0', '9', '6', '--']
['08/18', '120110', '462', '3459', '-1574.0', '14', '11', '--']
['08/18', '120110', '463', '4451', '-2593.0', '14', '15', '--']
['08/18', '120110', '464', '4773', '-3704.0', '16', '3', '--']
['08/18', '120110', '465', '1194', '694.0', '12', '1', '--']
['08/18', '120110', '466', '1685', '1667.0', '15', '10', '--']
['08/18', '120110', '467', '4894', '-417.0', '30', '12', '--']
['08/18'

['08/18', '120053', '585', '8559', '-93.0', '0', '21', '113']
['08/18', '120108', '586', '5827', '-1991.0', '0', '0', '27']
['08/18', '120121', '587', '5770', '-1111.0', '0', '11', '70']
['08/18', '120069', '588', '5401', '-1250.0', '0', '9', '26']
['08/18', '119984', '589', '4367', '-1528.0', '12', '10', '--']
['08/18', '120022', '590', '3842', '-2037.0', '16', '10', '--']
['08/18', '120022', '591', '4224', '-2546.0', '18', '4', '--']
['08/18', '120026', '592', '3864', '-926.0', '12', '9', '--']
['08/18', '120026', '593', '6212', '1898.0', '30', '17', '--']
['08/18', '120139', '594', '3800', '-2176.0', '0', '8', '16']
['08/18', '119940', '595', '5068', '-2130.0', '13', '0', '16']
['08/18', '120123', '596', '6040', '4213.0', '0', '11', '79']
['08/18', '120139', '597', '2459', '-1250.0', '0', '6', '11']
['08/18', '119897', '598', '3745', '278.0', '30', '0', '23']
['08/18', '119947', '599', '7112', '-2870.0', '0', '17', '26']
['08/18', '120127', '600', '8223', '-4954.0', '0', '0', '11']


{'spreadsheetId': '1SAAVEZNpyXmPW_gNKJZbcG-UC8m5IU2h-oaQqTyFz60',
 'updates': {'spreadsheetId': '1SAAVEZNpyXmPW_gNKJZbcG-UC8m5IU2h-oaQqTyFz60',
  'updatedRange': 'S5RAW!F7122:M7161',
  'updatedRows': 40,
  'updatedColumns': 8,
  'updatedCells': 320}}